### Data Analysis - Schadendaten aus GemDat 
Mirco Heidemann, 05/07/2018

In [1]:
import os as os
import numpy as np
import pandas as pd

/Users/mirco/miniconda3/envs/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mirco/miniconda3/envs/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
# get working directory
os.getcwd()

In [ ]:
# look at source csv file
# !head -n 3 data/A_Schadendatenper31.05.2018_9639.csv

In [ ]:
# read the CSV into a pandas data frame (df)
df = pd.read_csv("data/A_Schadendatenper31.05.2018_9639.csv", delimiter=';', encoding = "ISO-8859-1")

In [ ]:
# print first five rows, analog to R's head(df)
# df.head()

# pandas equivalents for R functions str()
# df.info()

#### Rename and arrange the pandas data frame, specify datatypes

In [ ]:
# rename columns: 'alter_name': 'neuer_name'
df.rename(columns={'GebaeudeId': 'geb_id',
                   'GebaeudeSuchbegriff': 'geb_nr',
                   'SchadenVersicherungWert': 'vers_sum',
                   'GebaeudeBaujahr': 'baujahr',
                   'GebaeudeGemeindeName': 'gemeinde',
                   'Ausdr2': 'gemeinde_id',
                   'GebaeudeZweckCode': 'zwk_code',
                   'GebaeudeZweckText': 'zwk_code_text',
                   'SchadenId': 'schaden_id',
                   'SchadenNr': 'schaden_nr',
                   'SchadenDatum': 'schaden_datum',
                   'SchadenSumme': 'schaden_sum',
                   'SchadenArtBezeichnung': 'schaden_art',
                   'CodTextDt': 'schaden_code',
                   'StcTextDt': 'status'},
          inplace = True) # rename the existing DataFrame (rather than creating a copy)

# converting schaden_datum to date format
df['schaden_datum'] = pd.to_datetime(df.schaden_datum)

In [ ]:
# selecting columns
# df_selected = df[['geb_id', 'schaden_id', 'schaden_datum']]

# write the new csv
# df_new.to_csv('schad_example_new.csv', index = False, encoding = "ISO-8859-1") # encoding = "utf-8"

In [ ]:
# dropping columns
df_schad = df.drop(columns=['Erstellung', 'Ausdr1', 'GebaeudeStrasse', 'GebaeudeHausNr', 'GebaeudePlz',
                          'GebaeudeOrt', 'Ausdr3', 'GebaeudeVolumen', 'EdiStcDis', 'EdiAfDat'])
# df_schad.info()
# df_schad.head()

####  Index losses

In [ ]:
# read index csv and rename colums
gvz_index = pd.read_csv("../GVZ_Portfolio/versicherungsindex_gvz.csv", sep = ';')

gvz_index = gvz_index.rename(columns={'Versicherungsindex GVZ': 'index',
                             'Jahr': 'jahr'})

gvz_index.jahr = pd.to_datetime(gvz_index.jahr)
gvz_index.jahr = gvz_index.jahr.dt.strftime('%Y')

In [ ]:
# year of loss as new column
df_schad['jahr'] = df_schad['schaden_datum'].dt.strftime('%Y')

# merge gvz-index to df_schad
df_schad = pd.merge(df_schad,
                 gvz_index[['jahr', 'index']],
                 on='jahr')

In [ ]:
# calculate indexed loss
df_schad['schaden_index'] = gvz_index['index'].max() / df_schad['index'] * df_schad['schaden_sum']
# df_schad.sort_values(by = "schaden_datum")

#### Select only elementar losses greater than zero and aggregate it to yearly losses for each peril (flood, hail, etc.)

In [ ]:
# Suppressing scientific notation in pandas
pd.options.display.float_format = '{:20,.0f}'.format

# only elementar losses greater than zero
df_elementar = df_schad[(df_schad['schaden_art'] == "Elementar") & (df_schad['schaden_sum'] > 0)]

# group by 'schaden_code' and 'jahr'
grouped = df_elementar.groupby(['schaden_code', 'jahr'])

# once the GroupBy object ('grouped') has been created, aggregate it
by_peril = grouped['schaden_index'].aggregate([np.sum, np.mean, np.median, np.max])

#### Selecting only losses from one peril and aggregate it to yearly losses

In [ ]:
# group by 'schaden_code'
grouped_peril = df_elementar.groupby('schaden_code')

# selecting a single group, eg. 'flood'
by_single_peril = grouped_peril.get_group('3 Hochwasser, Überschwemmung')

# group by 'jahr'
by_single_peril = by_single_peril.groupby('jahr')

# aggregate it to yearly losses
by_single_peril = by_single_peril['schaden_index'].aggregate([np.sum, np.size, np.mean, np.median, np.max])

#### Arrange df that jahr is the index and for each peril a column

In [ ]:
# group by 'schaden_code' and 'jahr'
grouped = df_elementar.groupby(['schaden_code', 'jahr'])

# aggregate 'schaden_index' per year, resetting indexes
by_jahr = grouped['schaden_index'].aggregate([np.sum]).reset_index()

# spread by schaden_code: index = jahr, columns = schaden_code, values = sum
by_jahr = by_jahr.pivot(index = 'jahr', columns = 'schaden_code', values='sum')

# rename columns
by_jahr = by_jahr.rename(columns={'.': 'Unbekannt', '1 Sturm': 'Sturm', '2 Hagel': 'Hagel',
                                  '3 Hochwasser, Überschwemmung': 'Flut',
                                  '4 Erdrutsch, Steinschlag': 'Erdrutsch_Steinschlag',
                                  '5 Schneedruck': 'Schneedruck', '6 Lawinen': 'Dachlawine'})

# replace NAs with zero
by_jahr.fillna(0, inplace=True)

# new categorie 'Rest' = Unbekannt + Erdrutsch_Steinschlag + Schneedruck + Dachlawine
by_jahr['Rest'] = by_jahr['Unbekannt'] + by_jahr['Erdrutsch_Steinschlag'] +\
by_jahr['Schneedruck'] + by_jahr['Dachlawine']

# dropping the unused columns
by_jahr = by_jahr.drop(columns=['Unbekannt', 'Erdrutsch_Steinschlag', 'Schneedruck', 'Dachlawine'])

#### Plot with Altair
Note: 'jahr' is the index and not a column of the grouped data frame

In [ ]:
import altair as alt
from vega_datasets import data

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')

In [ ]:
# reset index, make 'jahr' to a column in the data frame
df_by_jahr = by_jahr.reset_index()

Simple bar chart

In [ ]:
alt.Chart(df_by_jahr).mark_bar().encode(
    x='jahr',
    y='Hagel'
)

Stacked Bar Chart

In [ ]:
# reshaping df for the stacked bar chart
df_melt = df_by_jahr.melt(id_vars = 'jahr')

alt.Chart(df_melt).mark_bar().encode(
    # tell Altair which field to use for color segmentation 
    alt.Color('schaden_code',
        legend=alt.Legend(title='Schadenursache'),
        scale=alt.Scale(
            domain=['Sturm', 'Hagel', 'Flut', 'Rest'],
            range=['#e7ba42', '#c7c7c7', '#aec7e8', '#1f77b4']
        ),
    ),
    # tell Altair which field to group columns on
    alt.X('jahr',
        axis=alt.Axis(title='jahr'),
         ),
    # tell Altair which field to use as Y values and how to calculate
    alt.Y('sum(value)',
       axis=alt.Axis(title='indexierte Schadensumme'),
      ),
) #.properties(width=300, height=150)